<a href="https://colab.research.google.com/github/ShlokArora2709/Deep-Learning/blob/master/Code_GeN_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install transformers[torch] accelerate -U
!pip install PyGithub
!pip install datasets

In [16]:
from github import Github
import re
from datasets import Dataset

g=Github("")#github token API

AssertionError: 

In [3]:
repo=g.get_repo("openai/gym")

def extract_func(code):
  pattern=re.compile(r'def\s+(\w+)\s*\(.*\):')
  matches=pattern.findall(code)
  return matches

py_files=[]
contents= repo.get_contents("")
while contents:
  file_content=contents.pop(0)
  if file_content.type=="dir":
    contents.extend(repo.get_contents(file_content.path))
  elif file_content.name.endswith(".py"):
    py_files.append(file_content)

data = {"code": [], "function_name": []}

for file in py_files:
  code=file.decoded_content.decode("utf-8")
  func_names=extract_func(code)
  data["code"].extend([code]*len(func_names))
  data["function_name"].extend(func_names)

In [4]:
dataset=Dataset.from_dict(data)

dataset.save_to_disk("dataset")

Saving the dataset (0/1 shards):   0%|          | 0/974 [00:00<?, ? examples/s]

In [5]:
from datasets import load_from_disk

dataset=load_from_disk("dataset")
from transformers import AutoTokenizer,Trainer,TrainingArguments, AutoModelForCausalLM

tokenizer=AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model=AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

tokenizer.pad_token=tokenizer.eos_token

dataset=dataset.train_test_split(test_size=0.2)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def preprocess(code):
    tokenized_input = tokenizer(code["code"], truncation=True, padding="max_length")
    tokenized_input["labels"] = tokenized_input["input_ids"].copy()  # Use input_ids as labels
    return tokenized_input

In [7]:
import torch
tokenized_data=dataset.map(preprocess,batched=True)
model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    save_steps=1_000,
    save_total_limit=2,
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
)

torch.cuda.empty_cache()  # Clear GPU cache before training

trainer.train()

torch.cuda.empty_cache()  # Clear GPU cache after training

Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with `config.gradient_checkpointing=True`. Setting `use_cache=False`...


Step,Training Loss
500,0.286300


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [15]:
def generate_code(prompt, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length
    )
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

prompt = "def selection_sort(arr):"
generated_code = generate_code(prompt)

print("Generated Code:")
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Code:
def selection_sort(arr):
    """
    Implementation of the selection sort algorithm

    Args:
        arr: The array to sort

    Returns:
        The sorted array
    """
    for i in range(len(arr)):
        min_index = i
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min_index]:
                min_index = j

        arr[i], arr[min_index] = arr[min_index], arr[i]

    return arr


def test_selection_sort():
    arr = [3, 4, 5, 6, 7]


In [8]:
!nvidia-smi

Sat Aug 24 07:09:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              36W /  70W |   5663MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--